<a href="https://colab.research.google.com/github/LihiNofar/Veritas-/blob/main/Interview_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Veritas-Data-Set.csv to Veritas-Data-Set.csv


In [ ]:
import pandas as pd

df = pd.read_csv('Veritas-Data-Set.csv', encoding='ISO-8859-1')
df.shape

(82332, 45)

In [ ]:
import duckdb
query = """
SELECT proto, COUNT(*) AS event_count
FROM df
GROUP BY proto
ORDER BY event_count DESC;
"""

result1 = duckdb.query(query).df()
result1

,proto,event_count
0,tcp,43095
1,udp,29418
2,unas,3515
3,arp,987
4,ospf,676
...,...,...
126,sccopmce,32
127,iplt,32
128,pipe,32
129,ib,31


In [ ]:
query = """
SELECT
  proto,
  SUM(sbytes+dbytes) AS total_bytes,
  AVG(sbytes+dbytes) AS avg_bytes
FROM df
GROUP BY proto
ORDER BY total_bytes DESC;
"""

result2 = duckdb.query(query).df()
pd.options.display.float_format = '{:,.0f}'.format
result2


,proto,total_bytes,avg_bytes
0,tcp,"1,724,299,718","40,012"
1,udp,"10,350,834",352
2,sctp,"6,899,706","21,295"
3,ospf,"4,531,812","6,704"
4,unas,"703,000",200
...,...,...,...
126,ip,"5,940",180
127,ipnip,"5,940",180
128,egp,"5,940",180
129,nvp,"5,940",180


In [ ]:

query = """
SELECT
proto,sbytes,dbytes,
  TRY_CAST(sbytes AS DOUBLE) / (TRY_CAST(dbytes AS DOUBLE) + 1.0) AS ratio
FROM df
ORDER BY ratio DESC
LIMIT 5;
"""

result3 = duckdb.query(query).df()
result3


,proto,sbytes,dbytes,ratio
0,ospf,33728,0,"33,728.00"
1,ospf,33728,0,"33,728.00"
2,ospf,29920,0,"29,920.00"
3,ospf,29920,0,"29,920.00"
4,ospf,29920,0,"29,920.00"


In [ ]:
query = """
SELECT
  proto,
  SUM(TRY_CAST(sbytes AS DOUBLE) / (TRY_CAST(dbytes AS DOUBLE) + 1.0)) AS sum_ratio,
  AVG(TRY_CAST(sbytes AS DOUBLE) / (TRY_CAST(dbytes AS DOUBLE) + 1.0)) AS avg_ratio
FROM df
WHERE proto IN ('tcp', 'udp')
GROUP BY proto;
"""

result4 = duckdb.query(query).df()
result4

,proto,sum_ratio,avg_ratio
0,udp,"6,094,712.12",207.18
1,tcp,"578,622.12",13.43


In [ ]:
query = """
SELECT
  attack_cat,
  AVG (spkts) AS avg_spkts,
  AVG (dpkts) AS avg_dpkts
FROM df
GROUP BY attack_cat;
"""

result5 = duckdb.query(query).df()
result5

,attack_cat,avg_spkts,avg_dpkts
0,Backdoor,4.38,0.84
1,DoS,28.89,28.94
2,Exploits,37.70,27.02
3,Worms,16.77,57.32
4,Shellcode,6.06,3.36
5,Normal,22.78,25.42
6,Reconnaissance,6.97,4.11
7,Analysis,3.12,0.73
8,Fuzzers,11.79,5.79
9,Generic,2.80,1.64


In [ ]:
query = """
SELECT
  service,
  COUNT(*) AS attack_count
FROM df
WHERE label = 1
GROUP BY service
ORDER BY attack_count DESC
LIMIT 5;
"""

result6 = duckdb.query(query).df()
pd.options.display.float_format = '{:,.0f}'.format
result6


,service,attack_count
0,-,19778
1,dns,18299
2,http,4274
3,smtp,1216
4,ftp,794


In [ ]:
query = """
SELECT
  attack_cat,
  COUNT(*) AS number_of_events,
  AVG(rate) AS avg_rate,
  AVG(dur) AS avg_duration
FROM df
GROUP BY attack_cat
ORDER BY avg_rate DESC;
"""

result7 = duckdb.query(query).df()
pd.options.display.float_format = '{:,.2f}'.format
result7


,attack_cat,number_of_events,avg_rate,avg_duration
0,Generic,18871,"195,072.43",0.06
1,Analysis,677,"170,783.81",0.28
2,Backdoor,583,"154,631.49",0.92
3,DoS,4089,"129,943.21",2.07
4,Shellcode,378,"93,653.83",0.35
5,Reconnaissance,3496,"87,339.72",0.84
6,Fuzzers,6062,"67,530.25",2.12
7,Exploits,11132,"50,912.09",1.72
8,Normal,37000,"28,349.99",1.01
9,Worms,44,"20,921.26",1.05
